In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
print(tf.__version__)

2.13.0


Define a plot function

In [3]:
def plot_series(time, series, format="-", start=0, end=None):
    plt.plot(time[start:end], series[start:end], format)
    plt.xlabel("Time", fontsize=15)
    plt.ylabel("Disk Usages", fontsize=15)
    plt.grid(True)

In [ ]:
df = pd.read_csv("43.csv")
df

Dataset information

row[0]=time_step;
row[4]=Cpu_usages;
row[7]=RAM_usage;
row[8]=Disk_read;
row[9]=Disk_write;
row[11]=Bandwidth;

In [ ]:
Sum_disk=(df["Disk read throughput [KB/s]"] + df["Disk write throughput [KB/s]"])
df["Disk I/O"]=Sum_disk
df["Disk I/O"]=((df["Disk I/O"]-df["Disk I/O"].min())/(df["Disk I/O"].max()-df["Disk I/O"].min()))*20
df

In [ ]:
import csv
time_step = []
BW_usage = []

with open('43.csv') as csvfile:
  reader = csv.reader(csvfile, delimiter=',')
  next(reader)
  for row in reader:
    BW_usage.append(float(row[12]))
    time_step.append(int(row[0]))

In [ ]:
print(time_step[:10])
print(BW_usage[:10])

# Visualize Bandwidth usage data against time

In [ ]:
series = np.array(BW_usage)
time = np.array(time_step)
plt.figure(figsize=(10, 6))
#plt.title("Bandwidth Usages dataset",fontsize=16)

plot_series(time, series)

# Split dataset into train and test set

In [ ]:
split_time = 7500
time_train = time[:split_time]
x_train = series[:split_time]
time_valid = time[split_time:]
x_valid = series[split_time:]

Plot test data

In [ ]:
plt.figure(figsize=(10, 6))
#plt.title("Test Data",fontsize=16)
plot_series(time_valid, x_valid)

# window function definition

In [ ]:
#window function defination
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

In [ ]:
np.random.seed(51)
window_size = 64
batch_size = 125
shuffle_buffer_size = 1000
train_set = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)
print(train_set)

neural network model definition

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv1D(filters=64, kernel_size=5,
                      strides=1, padding="causal",
                      activation="relu",
                      input_shape=[None, 1]),
  tf.keras.layers.LSTM(64, return_sequences=True),
  tf.keras.layers.GRU(64, return_sequences=True),
  tf.keras.layers.Dense(30, activation="relu"),
  tf.keras.layers.Dense(15, activation="relu"),
  tf.keras.layers.Dense(1),
  tf.keras.layers.Lambda(lambda x: x * 100)
])

In [ ]:
lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-8 * 10**(epoch / 20))
optimizer = tf.keras.optimizers.SGD(lr=1e-8, momentum=0.9)
model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])
history = model.fit(train_set, epochs=100, callbacks=[lr_schedule])

Finding Minimum loss

In [ ]:
min_loss=min(history.history["loss"])
min_loss_index=history.history["loss"].index(min_loss)
min_lr=(history.history["lr"])[min_loss_index]
min_lr

Plotting minimum loss point

In [ ]:
plt.figure(figsize=(10, 6))
plt.semilogx(history.history["lr"], history.history["loss"],'r')
plt.axis([1e-8, 3e-2, 0, 50])
plt.xlabel("Learning rate", fontsize=15)
plt.ylabel("Loss", fontsize=15)
#plt.grid()
s='Min_lr: '+str(min_lr)
plt.axvline(x=min_lr, linewidth=2,color="b",ls="--")
plt.annotate(s, xy=(min_lr, min_loss), xytext=(1E-7, 25),arrowprops=dict(facecolor='black', shrink=0.05 ,width=1.0, headwidth=5.0),fontsize=15)
#plt.savefig('lr_vs_loss.png', dpi = 720) 
plt.show()